<a href="https://colab.research.google.com/github/KryssyCo/DS-Unit-2-Tree-Ensembles/blob/master/Krista_Shepard_DS5_S6M4_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment
- Learn about the mathematics of Logistic Regression by watching Aaron Gallant's [video #1](https://www.youtube.com/watch?v=pREaWFli-5I) (12 minutes) & [video #2](https://www.youtube.com/watch?v=bDQgVt4hFgY) (9 minutes). **COMPLETE**
- Start a clean notebook. **COMPLETE**
- Do train/validate/test split with the Tanzania Waterpumps data.**COMPLETE**
- Begin to explore and clean the data. For ideas, refer to [The Quartz guide to bad data](https://github.com/Quartz/bad-data-guide),  a "reference to problems seen in real-world data along with suggestions on how to resolve them." One of the issues is ["Zeros replace missing values."](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values) **COMPLETE**
- Select different numeric and categorical features. 
- Do one-hot encoding. (Remember it may not work with high cardinality categoricals.) **COMPLETE**
- Scale features. **COMPLETE**
- Use scikit-learn for logistic regression. **COMPLETE**
- Get your validation accuracy score. **COMPLETE**
- Get and plot your coefficients. **COMPLETE**
- Submit your predictions to our Kaggle competition.
- Commit your notebook to your fork of the GitHub repo.




In [0]:
#Import Tanzania Waterpumps data.
import pandas as pd

WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/tanzania/'

train_features = pd.read_csv(WEB + 'train_features.csv')
train_labels = pd.read_csv(WEB + 'train_labels.csv')
test_features = pd.read_csv(WEB + 'test_features.csv')
sample_submission = pd.read_csv(WEB + 'sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

In [17]:
# Core training data
train_features.describe() 

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [18]:
!pip install -U pandas-profiling

Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.6/dist-packages (2.1.2)


### Features

Your goal is to predict the operating condition of a waterpoint for each record in the dataset. You are provided the following set of information about the waterpoints:

- `amount_tsh` : Total static head (amount water available to waterpoint)
- `date_recorded` : The date the row was entered
- `funder` : Who funded the well
- `gps_height` : Altitude of the well
- `installer` : Organization that installed the well
- `longitude` : GPS coordinate
- `latitude` : GPS coordinate
- `wpt_name` : Name of the waterpoint if there is one
- `num_private` :  
- `basin` : Geographic water basin
- `subvillage` : Geographic location
- `region` : Geographic location
- `region_code` : Geographic location (coded)
- `district_code` : Geographic location (coded)
- `lga` : Geographic location
- `ward` : Geographic location
- `population` : Population around the well
- `public_meeting` : True/False
- `recorded_by` : Group entering this row of data
- `scheme_management` : Who operates the waterpoint
- `scheme_name` : Who operates the waterpoint
- `permit` : If the waterpoint is permitted
- `construction_year` : Year the waterpoint was constructed
- `extraction_type` : The kind of extraction the waterpoint uses
- `extraction_type_group` : The kind of extraction the waterpoint uses
- `extraction_type_class` : The kind of extraction the waterpoint uses
- `management` : How the waterpoint is managed
- `management_group` : How the waterpoint is managed
- `payment` : What the water costs
- `payment_type` : What the water costs
- `water_quality` : The quality of the water
- `quality_group` : The quality of the water
- `quantity` : The quantity of water
- `quantity_group` : The quantity of water
- `source` : The source of the water
- `source_type` : The source of the water
- `source_class` : The source of the water
- `waterpoint_type` : The kind of waterpoint
- `waterpoint_type_group` : The kind of waterpoint

### Labels

There are three possible values:

- `functional` : the waterpoint is operational and there are no repairs needed
- `functional needs repair` : the waterpoint is operational, but needs repairs
- `non functional` : the waterpoint is not operational

In [0]:
# Labels = three possible values:

# functional : the waterpoint is operational and there are no repairs needed
# functional needs repair : the waterpoint is operational, but needs repairs
# non functional : the waterpoint is not operational

In [20]:
train_labels.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [21]:
train_labels['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [22]:
test_features.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,14358.000000,14358.000000,14358.0000,14358.000000,1.435800e+04,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000
mean,37232.859799,324.219996,653.6363,34.082414,-5.697584e+00,0.408971,15.156359,5.713052,187.055439,1298.251985
std,21382.890432,2533.367778,688.2721,6.564449,2.947444e+00,8.231859,17.387588,9.794304,476.065978,952.551852
min,10.000000,0.000000,-57.0000,0.000000,-1.156459e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18765.500000,0.000000,0.0000,33.062317,-8.453125e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37442.000000,0.000000,346.0000,34.898976,-5.087905e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55909.250000,25.000000,1306.0000,37.221606,-3.314240e+00,0.000000,17.000000,5.000000,230.000000,2004.000000
max,74249.000000,200000.000000,2777.0000,40.325016,-2.000000e-08,669.000000,99.000000,80.000000,11469.000000,2013.000000


In [23]:
sample_submission.head()

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,functional
4,49871,functional


In [0]:
# import pandas_profiling
# train_features.profile_report()

In [24]:
# Train/Test Split for my Kaggle Competition
from sklearn.model_selection import train_test_split

X_train = train_features
y_train = train_labels['status_group']

X_train, X_val, y_train, y_val = train_test_split( X_train, y_train,
    train_size=0.70, test_size=0.30, stratify=y_train, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((41580, 40), (17820, 40), (41580,), (17820,))

In [25]:
#Double check starification
y_train.value_counts(normalize=True)

functional                 0.543074
non functional             0.384247
functional needs repair    0.072679
Name: status_group, dtype: float64

In [26]:
y_val.value_counts(normalize=True)

functional                 0.543098
non functional             0.384231
functional needs repair    0.072671
Name: status_group, dtype: float64

In [0]:
# fit a Logistic Regression using scikit-learn
# Begin with baselines: fast, first models

X_train_numeric = X_train.select_dtypes('number')
X_val_numeric = X_val.select_dtypes('number')

In [28]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=10000)
model.fit(X_train_numeric, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
#Evaluate on validation data
from sklearn.metrics import accuracy_score
y_pred=model.predict(X_val_numeric)
accuracy_score(y_val, y_pred)

0.5510662177328844

In [34]:
#Shorthand for scoring like above
model.score(X_val_numeric, y_val)

0.5510662177328844

In [35]:
# What predictions does a Logistic Regression return
y_pred

array(['functional', 'non functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [36]:
pd.Series(y_pred).value_counts()

functional        16148
non functional     1672
dtype: int64

In [37]:
#Check cardinality of categorical features
X_train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
recorded_by,41580,1,GeoData Consultants Ltd,41580
public_meeting,39283,2,True,35748
permit,39429,2,True,27190
source_class,41580,3,groundwater,32085
management_group,41580,5,user-group,36821
quantity_group,41580,5,enough,23232
quantity,41580,5,enough,23232
waterpoint_type_group,41580,6,communal standpipe,24183
quality_group,41580,6,good,35496
payment_type,41580,7,never pay,17792


In [38]:
#Exploring categorical features
X_train['payment'].value_counts()

never pay                17792
pay per bucket            6243
pay monthly               5819
unknown                   5696
pay when scheme fails     2718
pay annually              2558
other                      754
Name: payment, dtype: int64

In [39]:
#Exploring categorical features
X_train['quality_group'].value_counts()

good        35496
salty        3683
unknown      1315
milky         574
colored       351
fluoride      161
Name: quality_group, dtype: int64

In [40]:
#Exploring categorical features
X_train['waterpoint_type'].value_counts()

communal standpipe             19911
hand pump                      12274
other                           4480
communal standpipe multiple     4272
improved spring                  561
cattle trough                     79
dam                                3
Name: waterpoint_type, dtype: int64

In [41]:
#Encoding categorical feature
!pip install category_encoders

In [0]:
import category_encoders as ce

In [43]:
encoder = ce.OneHotEncoder(use_cat_names=True)
encoded = encoder.fit_transform(X_train['waterpoint_type'])
encoded.head()

,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_hand pump,waterpoint_type_other,waterpoint_type_improved spring,waterpoint_type_cattle trough,waterpoint_type_dam
5974,1,0,0,0,0,0,0
31787,0,1,0,0,0,0,0
27800,1,0,0,0,0,0,0
8008,0,1,0,0,0,0,0
39725,0,1,0,0,0,0,0


In [44]:
# How to use encoders and scalers in scikit-learn
# Use the fit_transform method on the train set
# Use the transform method on the validation set

from sklearn.preprocessing import StandardScaler

categorical_features = ['waterpoint_type']
numeric_features = X_train.select_dtypes('number').columns.drop('id').tolist()
features = categorical_features + numeric_features

X_train_subset = X_train[features]
X_val_subset = X_val[features]

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train_subset)
X_val_encoded = encoder.transform(X_val_subset)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)

model = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)
model.fit(X_train_scaled, y_train)
print('Validation Accuracy', model.score(X_val_scaled, y_val))

Validation Accuracy 0.6303030303030303


In [0]:
# Compare original features, encoded features, & scaled features
print(X_train.shape)
X_train[:1].T


In [0]:
X_train_scaled.shape

In [0]:
X_train_scaled[:1].T

In [0]:
#Get and plot coefficients
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))

coefficients = pd.Series(model.coef_[0], X_train_encoded.columns)
coefficients.sort_values().plot.barh();

In [51]:
X_test_subset = test_features[features]
X_test_encoded = encoder.transform(X_test_subset)
X_test_scaled = scaler.transform(X_test_encoded)
all(X_test_encoded.columns == X_train_encoded.columns)

True

In [0]:
y_pred = model.predict(X_test_scaled)
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-01.csv', index=False)

In [0]:
from google.colab import files
files.download('submission-01.csv')

## Stretch Goals
- Begin to visualize the data.
- Try different [scikit-learn scalers](https://scikit-learn.org/stable/modules/preprocessing.html)
- Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html):

> Pipeline can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification. Pipeline serves multiple purposes here:

> - **Convenience and encapsulation.** You only have to call fit and predict once on your data to fit a whole sequence of estimators.
> - **Joint parameter selection.** You can grid search over parameters of all estimators in the pipeline at once.
> - **Safety.** Pipelines help avoid leaking statistics from your test data into the trained model in cross-validation, by ensuring that the same samples are used to train the transformers and predictors.